In [1]:
import awkward as ak
import uproot
import numpy as np
import pandas as pd

In [2]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema

In [3]:
signaldomain = '/ecoderemdvol/econ_ntuples/qk/econ_ntuple_40_particles.root'
signalfile = uproot.open(signaldomain)

In [4]:
events = NanoEventsFactory.from_root(
    signalfile,
    treepath='FloatingpointThreshold0DummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

In [5]:
gen_pt = events.gen_pt
gen_eta = events.gen_eta

In [6]:
len(gen_eta)

20

In [10]:
pt_mask = (gen_pt <=50)[:,0]
eta_min_mask = (gen_eta[:,0] >=2.1)
eta_max_mask = (gen_eta[:,0] <= 2.7)

mask = ak.Array(np.logical_and(np.asarray(pt_mask),np.asarray(eta_min_mask),np.asarray(eta_max_mask)))

In [11]:
test = (mask == [False]*len(mask))

In [12]:
ak.all(test)

False

In [13]:
tc_mask_data = ak.Array(
    {
        'tc_zside': events['tc_zside'][mask],
        'tc_layer': events['tc_layer'][mask],
        'tc_waferu': events['tc_waferu'][mask],
        'tc_waferv': events['tc_waferv'][mask],
        'tc_cellu': events['tc_cellu'][mask],
        'tc_cellv': events['tc_cellv'][mask],
        'tc_data': events['tc_data'][mask],
        'tc_simenergy': events['tc_simenergy'][mask],
        'tc_eta': events['tc_eta'][mask]
    }                
)

In [14]:
select_eLinks = {5 : (tc_mask_data[b'tc_layer']==9),
                4 : (tc_mask_data[b'tc_layer']==7) | (tc_mask_data[b'tc_layer']==11),
                             3 : (tc_mask_data[b'tc_layer']==13),
                             2 : (tc_mask_data[b'tc_layer']<7) | (tc_mask_data[b'tc_layer']>13),
                             -1 : (tc_mask_data[b'tc_layer']>0)}

assert 5 in select_eLinks

data = tc_mask_data[select_eLinks[5]]

In [15]:
df = ak.to_pandas(data)

In [17]:
dfRemap = pd.read_csv('/ecoderemdvol/tcRemap.csv')
df = df.reset_index().merge(dfRemap)

df['ADCT'] = (df.tc_data* ((1./np.cosh(df.tc_eta))*2**12).astype(int)/2**12).astype(int)

#create new unique index (can't get pivot working on multi-indexing, but this is essentially the same)
df['WaferEntryIdx'] = (df.entry*1000000 + df.tc_layer*10000 + df.tc_waferu*100 + df.tc_waferv)*df.tc_zside

val = 'ADCT'
dfTrainData = df.pivot_table(index='WaferEntryIdx',columns='tc_cell_train',values=val).fillna(0).astype(int)
dfTrainData.columns = [f'CALQ_{i}' for i in range(48)]

dfTrainData[['entry','zside','layer','waferu','waferv']] = df.groupby(['WaferEntryIdx'])[['entry','tc_zside','tc_layer','tc_waferu','tc_waferv']].mean()

dfTrainData['simenergy'] = df.groupby(['WaferEntryIdx'])[['tc_simenergy']].sum()

#Mapping wafer_u,v to physical coordinates
dfEtaPhi=pd.read_csv('/ecoderemdvol/WaferEtaPhiMap.csv')
dfTrainData=dfTrainData.merge(dfEtaPhi, on=['layer','waferu','waferv'])
dfTrainData.reset_index(drop=True,inplace=True)

In [18]:
dfTrainData

,CALQ_0,CALQ_1,CALQ_2,CALQ_3,CALQ_4,CALQ_5,CALQ_6,CALQ_7,CALQ_8,CALQ_9,...,CALQ_47,entry,zside,layer,waferu,waferv,simenergy,subdet,tc_eta,tc_phi
0,0,0,0,0,0,0,0,0,0,0,...,0,0.0,-1.0,9.0,10.0,8.0,0.0,1,1.531410,2.285104
1,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,9.0,10.0,8.0,0.0,1,1.531410,2.285104
2,0,0,0,0,0,0,0,0,0,35,...,0,0.0,-1.0,9.0,10.0,7.0,0.0,1,1.547228,2.395938
3,16,0,0,0,0,0,0,0,0,0,...,0,0.0,-1.0,9.0,10.0,6.0,0.0,1,1.573241,2.508065
4,0,0,0,12,0,0,0,0,0,0,...,9,0.0,1.0,9.0,10.0,6.0,0.0,1,1.573241,2.508065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,0,0,0,0,0,0,0,16,0,0,...,0,0.0,1.0,9.0,8.0,-2.0,0.0,1,1.540183,-2.967796
590,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,9.0,8.0,10.0,0.0,1,1.529415,1.881010
591,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,9.0,9.0,0.0,0.0,1,1.547188,0.414760
592,0,0,0,0,0,0,0,0,0,0,...,0,0.0,1.0,9.0,9.0,1.0,0.0,1,1.598238,3.036991
